# Jupyter Notebook to tune the hyperparameters of a model

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import kerastuner as kt

np.random.seed(42)
tf.random.set_seed(42)

2021-12-15 15:47:46.939252: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/tmp/ipykernel_10466/202277407.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


### lists possible devices (CPU, GPU), used to check if GPU is recognized/exists

In [2]:
tf.config.get_visible_devices()

2021-12-15 15:47:50.400976: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-15 15:47:50.401785: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-15 15:47:50.451936: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-15 15:47:50.452091: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.8225GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2021-12-15 15:47:50.452105: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-12-15 15:47:50.453315: I tensorflow/stream_executor/platform/d

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

#### preprocessing of the images applied when loading image data set from disk with tensorflows flow_from_directory

In [3]:
image_gen = keras.preprocessing.image.ImageDataGenerator(rotation_range=20,  # rotate the image 20 degrees
                                                         width_shift_range=0.2,
                                                         height_shift_range=0.2,
                                                         rescale=1 / 255,  # Rescale the image by normalzing it.
                                                         shear_range=0.15,
                                                         # Shear means cutting away part of the image (max 20%)
                                                         zoom_range=0.15,  # Zoom in by 15% max
                                                         horizontal_flip=True,  # Allow horizontal flipping
                                                         fill_mode='nearest'
                                                         # Fill in missing pixels with the nearest filled value
                                                         )

In [4]:
# path to the image data set

train_data_path = './data/data_heavily_reduced/data_balanced/train'  #local notebook
test_data_path = './data/data_heavily_reduced/data_balanced/test'  #local notebook
validation_data_path = './data/data_heavily_reduced/data_balanced/val'  #local notebook


In [5]:
# generate training set by loading the images from their directories with flow_from_directory
# important: the folder structure has to match! i.e {train} -> {ok,def}
# at the "same time" the data augmentation is applied on the images through the ImageDataGenerator
batch_size=14
train_image_gen = image_gen.flow_from_directory(train_data_path,
                                                target_size=(224, 224),
                                                batch_size=batch_size,
                                                class_mode='binary')

Found 7999 images belonging to 2 classes.


In [6]:
# generate validation set by loading the images from their directories with flow_from_directory
# important: the folder structure has to match! i.e {validation} -> {ok,def}
# at the "same time" the data augmentation is applied on the images through the ImageDataGenerator

valid_image_gen = image_gen.flow_from_directory(validation_data_path,
                                          target_size=(224,224),
                                          batch_size=batch_size,
                                          class_mode='binary')

Found 999 images belonging to 2 classes.


In [7]:
# generate test set by loading the images from their directories with flow_from_directory
# important: the folder structure has to match! i.e {test} -> {ok,def}
# at the "same time" the data augmentation is applied on the images through the ImageDataGenerator

test_image_gen = image_gen.flow_from_directory(test_data_path,
                                               target_size=(224,224),
                                               batch_size=batch_size,
                                               class_mode='binary')

Found 1001 images belonging to 2 classes.


In [8]:
# saves the model and its weights as a json file in the folder saved_models

def save_model(model, model_name):
    my_model = model.to_json()
    with open(f'./saved_models/{model_name}.json', "w") as file:
        file.write(my_model)
    # serialize weights to HDF5
    model.save_weights(f'./saved_model/{model_name}_weights.h5')

### used to load saved model with its weights

In [9]:
def load_model(model_path, weight_path):
    # load json and create model
    file = open(model_path, 'r')
    model_json = file.read()
    file.close()
    loaded_model = keras.models.model_from_json(model_json)
    # load weights
    loaded_model.load_weights(weight_path)
    optimizer = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
    loaded_model.compile(loss="binary_crossentropy", optimizer=optimizer,
                         metrics=['accuracy', 'Recall', 'Precision', 'AUC'])
    return loaded_model

### used to build the base model using predefined architectures
currently: vgg16, xception, resnet

In [10]:
def build_base_model(architecture, weights):
    input = tf.keras.Input(shape=(224, 224, 3))
    if architecture == 'vgg16':
        return tf.keras.applications.vgg16.VGG16(weights=weights, include_top=False, input_tensor=input)
    if architecture == 'xception':
        return tf.keras.applications.xception.Xception(weights=weights, include_top=False, input_tensor=input)
    if architecture == 'resnet':
        return tf.keras.applications.resnet.ResNet50(weights=weights, include_top=False, input_tensor=input)

### builds the model for hyperparameter tuning of the first pretraining step

In [12]:
def build_model(hp):
    base_model = build_base_model('vgg16', 'imagenet')
    flat = keras.layers.Flatten(name='flatten')(base_model.output)
    dense_1 = keras.layers.Dense(units=hp.Int('units', min_value=1000, max_value=4000, step=200))(flat)
    dropout = keras.layers.Dropout(rate=hp.Float('dropout_1',min_value=0.0,max_value=0.5,default=0.25,step=0.05,))(dense_1)
    batch = keras.layers.BatchNormalization()(dropout)
    output = keras.layers.Dense(1, activation='sigmoid')(batch)
    model = tf.keras.Model(base_model.input, output)
    for layer in base_model.layers:
        layer.trainable = False

    learning_rate = hp.Float('learning_rate',min_value=1e-5,max_value=1e-2,sampling='LOG',default=1e-3)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    return model

### defines the tuner

In [14]:
tuner = kt.Hyperband(
    hypermodel=build_model,
    hyperband_iterations=2,
    objective='val_accuracy',
    seed=42,
    executions_per_trial=1,
    directory='vgg16_tuner',
    project_name='first_training_step'
)

INFO:tensorflow:Reloading Oracle from existing project vgg16_tuner/first_training_step/oracle.json
INFO:tensorflow:Reloading Tuner from vgg16_tuner/first_training_step/tuner0.json


### uses the tuner with the defined parameters and defined search spaces to find the best parameter

In [15]:
tuner.search(train_image_gen, epochs=5, validation_data=valid_image_gen)

Trial 233 Complete [00h 26m 34s]
val_accuracy: 0.9789789915084839

Best val_accuracy So Far: 0.9909909963607788
Total elapsed time: 10h 17m 46s

Search: Running Trial #234

Hyperparameter    |Value             |Best Value So Far 
units             |3000              |2800              
dropout_1         |0.05              |0.25              
learning_rate     |1.1401e-05        |8.837e-05         
tuner/epochs      |34                |100               
tuner/initial_e...|12                |34                
tuner/bracket     |2                 |4                 
tuner/round       |1                 |4                 
tuner/trial_id    |39a4b0e51cdd189...|d805fa3e3d8af65...

Epoch 13/34
572/572 [==============================] - 72s 125ms/step - loss: 0.3597 - accuracy: 0.8567 - val_loss: 0.1410 - val_accuracy: 0.9459
Epoch 14/34
572/572 [==============================] - 71s 125ms/step - loss: 0.2059 - accuracy: 0.9275 - val_loss: 0.1731 - val_accuracy: 0.9389
Epoch 15/34
572/572 [

KeyboardInterrupt: 

### get the bet hyperparameters found by the tuner

In [ ]:
# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)

## build model for finding best hyperparameters for second pretraining step

In [19]:
def call_existing_code(lr, beta_1, beta_2):
    model = load_model('saved_models/vgg16_pretrained.json', 'saved_models/vgg16_pretrained_weights.h5')
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr, beta_1=beta_1, beta_2=beta_2),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    return model


def build_model(hp):
    lr = hp.Float('learning_rate',min_value=1e-5,max_value=1e-2,sampling='LOG',default=1e-3)
    beta_1 = hp.Float("beta_1", min_value=0.5, max_value=0.9, step=0.1, default=0.9)
    beta_2 = hp.Float("beta_2", min_value=0.800, max_value=0.999, step=0.001, default=0.999)
    model = call_existing_code(lr=lr, beta_1=beta_1, beta_2=beta_2)
    return model


build_model(kt.HyperParameters())

2021-12-09 23:12:55.093691: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-09 23:12:55.094129: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-09 23:12:55.094320: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.8225GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2021-12-09 23:12:55.094362: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library li

### define tuner

In [23]:
tuner = kt.Hyperband(
    hypermodel=build_model,
    objective="val_accuracy",
    hyperband_iterations=2,
    executions_per_trial=1,
    directory="vgg16_tuner",
    project_name="second_training_step",
)

### define early stoping callback

In [25]:
custom_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

2021-12-09 23:28:32.495167: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-12-09 23:28:32.495192: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-12-09 23:28:32.495431: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1365] Profiler found 1 GPUs
2021-12-09 23:28:32.503183: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcupti.so.10.1
2021-12-09 23:28:32.604357: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1415] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2021-12-09 23:28:32.604476: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


### look for the best hyperparameters

In [26]:
tuner.search(train_image_gen, epochs=100, validation_data=valid_image_gen, callbacks=[custom_early_stopping])

Trial 165 Complete [00h 04m 54s]
val_accuracy: 0.9859859943389893

Best val_accuracy So Far: 0.9949949979782104
Total elapsed time: 12h 16m 27s

Search: Running Trial #166

Hyperparameter    |Value             |Best Value So Far 
lr                |0.0075799         |0.00035095        
beta_1            |0.7               |0.8               
beta_2            |0.895             |0.844             
tuner/epochs      |4                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |3                 |4                 
tuner/round       |0                 |0                 

Epoch 1/4
  2/572 [..............................] - ETA: 1:10 - loss: 0.0084 - accuracy: 1.0000

2021-12-10 11:45:03.257047: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-12-10 11:45:03.257065: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-12-10 11:45:03.257105: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1415] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_NOT_INITIALIZED
2021-12-10 11:45:03.408065: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-12-10 11:45:03.409359: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:228]  GpuTracer has collected 0 callback api events and 0 activity events. 
2021-12-10 11:45:03.409947: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-12-10 11:45:03.410609: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/4c5617ae152dd85e9c041f35b3dc3a7c/execution0

572/572 [==============================] - 72s 124ms/step - loss: 0.1462 - accuracy: 0.9617 - val_loss: 0.1063 - val_accuracy: 0.9640
Epoch 2/4
572/572 [==============================] - 71s 124ms/step - loss: 0.1192 - accuracy: 0.9621 - val_loss: 0.0795 - val_accuracy: 0.9770
Epoch 3/4
555/572 [============================>.] - ETA: 1s - loss: 0.0862 - accuracy: 0.9704

KeyboardInterrupt: 

In [16]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 224, 224))
best_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [29]:
tuner.results_summary()

Results summary
Results in hp_tuner/run3
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
lr: 0.00035094611635709755
beta_1: 0.7999999999999999
beta_2: 0.8440000000000001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.9949949979782104
Trial summary
Hyperparameters:
lr: 0.00036937225014755195
beta_1: 0.8999999999999999
beta_2: 0.9590000000000002
tuner/epochs: 34
tuner/initial_epoch: 12
tuner/bracket: 4
tuner/round: 3
tuner/trial_id: 55a43167e971016004cb478df12e1b69
Score: 0.9939939975738525
Trial summary
Hyperparameters:
lr: 0.0005154994394291512
beta_1: 0.8999999999999999
beta_2: 0.9150000000000001
tuner/epochs: 34
tuner/initial_epoch: 12
tuner/bracket: 4
tuner/round: 3
tuner/trial_id: ef2e139c3a403f908682720e74f63ac1
Score: 0.9929929971694946
Trial summary
Hyperparameters:
lr: 0.00036937225014755195
beta_1: 0.8999999999999999
beta_2: 0.9590000000000002
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/

In [11]:
# Get the optimal hyperparameters
# best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
#
# print(f"""
# {best_hps.get('units')} is {best_hps.get('learning_rate')}. {best_hps.get('dropout_1')}
# """)